# Create Prediction Forecast Endpointd
1. Load CSV 
2. Create and Test Pickle
3. Create Prediction Function
4. Create Visualization Function
5. Test Prediction and Visualization Function

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from fbprophet import Prophet

## 1. Load CSV

In [ ]:
population_melt = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_melt.csv')
population = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_cleaned.csv')

## 2. Create and Test Pickle


### Create a dictionary
- zip city list
- create a list of grouped dataframe

In [ ]:
cities_list = list(population['City,State'])

### Fit FB Prophet model
- fit model on dataframe
- create multiple models 
- pickle models 

was unable to complete this process. The model took too long to even pickle for it to be useful as a model. 

### Fit model on DataFrame

In [ ]:
def make_model(start:0, end:100):
  cities_list = list(population['City,State'])[start:end]

  models = []
  for city_name in cities_list:
      df_ = population_melt[population_melt['City,State'] == city_name]
      m = Prophet(interval_width=0.95)
      model = m.fit(df_)
      models.append([city_name, model])

  # Putting information into dataframe
  model_df = pd.DataFrame(models, columns = ["cityname", 'model'])
  return model_df

In [ ]:
# model_0_100 = make_model(0, 100)

In [ ]:
# model_101_200 = make_model(101, 200)

In [ ]:
# model_201_300 = make_model(201, 300)

In [ ]:
# model_301_400 = make_model(301, 400)

In [ ]:
# model_401_500 = make_model(401, 500)

In [ ]:
# model_501_600 = make_model(501, 600)

In [ ]:
# model_601_700 = make_model(601, 700)

In [ ]:
# model_701_730 = make_model(701, 730)

### Create pickle of models

In [ ]:
# create list of models
model_list = [model_0_100, 
              model_101_200, 
              model_201_300, 
              model_301_400, 
              model_401_500, 
              model_601_700, 
              model_701_730]

for i in range(len(model_list)):
    with open(f'pickle/str{model_list[i]}', 'wb') as mod:
        pickle.dump(model_list[i], mod)

### Open Models

In [ ]:
# Access the models
df_100 = pickle.load(open("pickle/model_0_100.pkl", 'rb'))
df_200 = pickle.load(open("pickle/model_101_200.pkl", 'rb'))
df_300 = pickle.load(open("pickle/model_201_300.pkl", 'rb'))
df_400 = pickle.load(open("pickle/model_301_400.pkl", 'rb'))
df_500 = pickle.load(open("pickle/model_401_500.pkl", 'rb'))
df_600 = pickle.load(open("pickle/model_501_600.pkl", 'rb'))
df_700 = pickle.load(open("pickle/model_601_700.pkl", 'rb'))
df_730 = pickle.load(open("pickle/model_701_730.pkl", 'rb'))

# Concatenate the dataframes of zip codes/model
model_df_all= pd.concat([df_100, df_200, df_300, df_400, df_500, df_600, df_700, df_730])

### Serializing with json

In [ ]:
# Python
import json
from fbprophet.serialize import model_to_json, model_from_json

In [ ]:
for i in range(len(model_list)):
    with open('serialized_model.json', 'w') as fout:
        json.dump(model_to_json(model_list), fout)  # Save model

In [ ]:
with open('serialized_model.json', 'r') as fin:
    all_models = model_from_json(json.load(fin))  # Load model

## 3. Create Prediction and Visualization Function

In [ ]:
# this model works but just locally
def population_forecast(city, periods):
    # Load Dataset
  population = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_cleaned.csv')
  population.reset_index(level=0, inplace=True)

  # Melt table into ds and y
  population_melt = population[['City,State', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]
  population_melt = population_melt.melt(id_vars=['City,State'], var_name='ds', value_name='y')

  # Isolate city data
  city = [city]
  df_ = population_melt.loc[population_melt['City,State'].isin(city)][['ds', 'y']]
  print(df_)
  df_.columns = ['ds','y']

  # Fit and Predict on city dataframe
  # m = Prophet(interval_width=0.95)
  m.fit(df_)
  future = m.make_future_dataframe(periods=periods, freq='Y')
  forecast = m.predict(future)
  predictions = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][9:]
  predictions['ds'] = predictions['ds'].dt.year
  print(predictions['ds'])
  predictions[['yhat', 'yhat_lower', 'yhat_upper']] =  predictions[['yhat', 'yhat_lower', 'yhat_upper']].round()
  print(predictions.tail())

  # Create graph
  # Graph first 10 years
  df_['ds'] = df_['ds'].astype(int)
  ax = df_.plot(x = 'ds', y = 'y', label='Observed', figsize= (10, 8)) 

  # Graph predictions including the upper and lower bounds
  predictions['ds'] = predictions['ds'].astype(int)
  predictions[['ds', 'yhat']].plot(ax = ax, x = 'ds', y = 'yhat', label = "Forecast") 
  ax.fill_between(predictions['ds'],
                predictions['yhat_lower'],
                predictions['yhat_upper'],
                color='k', 
                alpha=.25)

  ax.set_xlabel('Year')
  ax.set_ylabel('Population')
  plt.title(f"{city[0]} Population" )
  plt.legend()

  return plt.show()

## 4. Create Visualization Function
- using saved predictions

In [ ]:
POPULATION_CSV = 'https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_cleaned.csv'
FORECAST_CSV = 'https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_prediction.csv'

def get_plot(city):
  city = [city]

  # Historical population data
  population = pd.read_csv(POPULATION_CSV)
  population = population[population['City,State'].isin(city)]
  population = population[['City,State', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]
  population_melt = population.melt(id_vars=['City,State'], var_name='ds', value_name='y')
  population_melt['ds'] = population_melt['ds'].astype(int)

  # Predictions
  forecast = pd.read_csv(FORECAST_CSV)
  df = forecast[forecast['City,State'].isin(city)][9:]
  df['year'] = df['year'].astype(int)

  # Graph Data
  ax = population_melt.plot(x = 'ds', y = 'y', label='Observed', figsize= (10, 8))
  df[['year', 'yhat']].plot(ax = ax, x = 'year', y = 'yhat', label = "Forecast")

  # Fill to show upper and lower bounds
  ax.fill_between(df['year'],
                df['yhat_lower'],
                df['yhat_upper'],
                color='k', 
                alpha=.25)

  ax.set_xlabel('Year')
  ax.set_ylabel('Population')
  plt.title(f"{city[0]} Population" )
  plt.legend()

  plt.show()

In [ ]:

# @router.post('/api/population_forecast')
# def population_forecast(city:City, periods=10):
#     """
#     Create visualization of historical and forecasted population

#     args:
#     - city: str -> The target city
#     - periods: int -> number of years to forecast for

#     Returns:
#     Visualization of population forecast
#     - 10 year of historical data
#     - forecasts for number of years entered
#     """

#     city = validate_city(city)

#     # Load Dataset
#     population = pd.read_csv('https://raw.githubusercontent.com/jiobu1/labspt15-cityspire-g-ds/main/notebooks/model/population2010-2019/csv/population_cleaned.csv')
#     population.reset_index(level=0, inplace=True)

#     # Melt table into ds and y
#     population_melt = population[['City,State', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]
#     population_melt = population_melt.melt(id_vars=['City,State'], var_name='ds', value_name='y')

#     # Isolate city data
#     location = [city.city + ', ' + city.state]
#     df_ = population_melt.loc[population_melt['City,State'].isin(location)][['ds','y']]
#     df_.columns = ['ds','y']


#     # Fit and Predict on city dataframe
#     # Model
#     with open("app/data/pickle_model/model.pkl", "rb") as f:
#         m = load(f)
#     m = Prophet(interval_width=0.95)

#     # Fit model
#     m.fit(df_)
#     future = m.make_future_dataframe(periods=periods, freq='Y')

#     # Predict
#     forecast = m.predict(future)
#     predictions = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][9:]
#     predictions['ds'] = pd.DatetimeIndex(predictions['ds']).year
#     predictions[['yhat', 'yhat_lower', 'yhat_upper']] =  predictions[['yhat', 'yhat_lower', 'yhat_upper']].round()

#     # Create graph
#     # Graph first 10 years
#     df_['ds'] = df_['ds'].astype(int)
#     predictions['ds'] = predictions['ds'].astype(int)

#     # Graph historical data
#     fig = go.Figure()

#     fig.add_trace(go.Scatter(
#         name = 'Original',
#         x = list(df_['ds']),
#         y = list(df_['y']),
#         fill = None,
#         mode = 'lines',
#         line_color = 'black',
#         showlegend = True
#     ))

#     # Graph predictions including the upper and lower bounds
#     fig.add_trace(go.Scatter(
#         name = 'Forecast',
#         x = list(predictions['ds']),
#         y = list(predictions['yhat']),
#         fill = None,
#         mode = 'lines',
#         line_color = 'red',
#         showlegend = True
#     ))

#     fig.add_trace(go.Scatter(
#         name = 'Lower Bound',
#         x = list(predictions['ds']),
#         y = list(predictions['yhat_lower']),
#         fill = None,
#         mode = 'lines',
#         line_color = 'gray',
#     ))

#     fig.add_trace(go.Scatter(
#         name = 'Upper Bound',
#         x = list(predictions['ds']),
#         y = list(predictions['yhat_upper']),
#         fill='tonexty',
#         mode='lines',
#         line_color = 'gray',
#     ))

#     # Edit the layout
#     fig.update_layout({
#         'autosize':True,
#         'title': f'{city[0]} Population Forecast',
#         'title_x': 0.5,
#         'xaxis_title': 'Year',
#         'yaxis_title': 'Population'
#         })

#     fig.update_yaxes(automargin = True,)
#     fig.update_xaxes(automargin = True, nticks=20)

#     fig.show()

#     return fig.to_json